In [102]:

import matplotlib.pyplot as plt
import pandas as pd
import pickle
import seaborn as sns
from pyprojroot import here


## Define policy category lists

In [103]:
all_policies = ['Paris Agreement', 'Green New Deal', 'Executive action', 'Emergency declaration',
                  '(De)regulation and laws', 'Renewable energy', 'Emission reduction',
                  'Oil and gas industry', 'Other climate policy']

no_other_policies = ['Paris Agreement', 'Green New Deal', 'Executive action', 'Emergency declaration',
                  '(De)regulation and laws', 'Renewable energy', 'Emission reduction',
                  'Oil and gas industry']

only_concrete_policies= ['Paris Agreement', 'Green New Deal', 'Emergency declaration',
                  'Renewable energy', 'Emission reduction',
                  'Oil and gas industry', 'Other climate policy']

regulation_col = '(De)regulation and laws'
executive_action_col = 'Executive action'


## Import data

In [104]:

# Read the annotation JSON file into a DataFrame
complete_df = pd.read_json(here("data/individual_policy_data/policy-recoding-2025-05-21-complete.json"))
changes_df =  pd.read_csv(here("data/individual_policy_data/KL_JOG_cleanup_changes.csv"))
Q3_df = pd.read_csv(here("data/processed_Q3_df/Q3_df.csv"))


In [105]:
Q3_df = Q3_df[Q3_df['policy'] != 'Does not mention'].copy()


In [106]:
# Count the number of times each internal_id appears
id_counts = Q3_df['internal_id'].value_counts()

# Count how many appear once vs. twice
transcript_annotation_counts = id_counts.value_counts().sort_index()

# Display results
print("Number of transcripts annotated once:", transcript_annotation_counts.get(1, 0))
print("Number of transcripts annotated twice:", transcript_annotation_counts.get(2, 0))
print("Number of transcripts annotated thrice:", transcript_annotation_counts.get(3, 0))


Number of transcripts annotated once: 119
Number of transcripts annotated twice: 629
Number of transcripts annotated thrice: 2


## Reformat data
This notebook reformats the data with post-hoc coding updates.

In [107]:
Q3_grouped = Q3_df.groupby(['internal_id', 'media_outlet'], as_index=False).agg({
    'policy': list
})

In [108]:
# Normalize policy column to always be a list
def standardize_policy(entry):
    if isinstance(entry, dict) and 'choices' in entry:
        return entry['choices']
    elif isinstance(entry, str):
        return [entry]
    elif isinstance(entry, list):
        return entry
    else:
        return []


# Apply normalization
complete_df['policy'] = complete_df['policy'].apply(standardize_policy)
df = complete_df[['internal_id','annotator','policy']]

In [109]:
# One-hot encode each policy annotation
def one_hot_encode(policies, all_policies):
    return [1 if policy in policies else 0 for policy in all_policies]

df["policy_encoded"] = df["policy"].apply(lambda x: one_hot_encode(x, all_policies))

/var/folders/v0/mrdbnbz979z112bjxxhzr3zh0000gn/T/ipykernel_31029/2247853040.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["policy_encoded"] = df["policy"].apply(lambda x: one_hot_encode(x, all_policies))


In [110]:
policy_df = pd.DataFrame(df["policy_encoded"].tolist(), columns=all_policies)
df_binary = pd.concat([df.drop(columns=["policy_encoded"]), policy_df], axis=1)
df_binary.to_csv("output/policy_category_df_binary.csv", index=False)

In [111]:
merged_df = pd.merge(Q3_df, df_binary, on='internal_id', how='inner')
merged_df = merged_df.rename(columns={'policy_x': 'response'})
len(merged_df)

2766

In [112]:

changes_df_filled = changes_df.fillna(0)

# Identify internal_ids to update
ids_to_update = changes_df_filled['internal_id'].unique()
print(f"Number of unique internal_ids to update: {len(ids_to_update)}")

#  Remove those rows from merged_df
filtered_merged_df = merged_df[~merged_df['internal_id'].isin(ids_to_update)].copy()

# Create replacement rows from changes_df
policy_columns = ['Paris Agreement', 'Green New Deal', 'Executive action', 
                  'Emergency declaration', '(De)regulation and laws', 
                  'Renewable energy', 'Emission reduction', 
                  'Oil and gas industry', 'Other climate policy']

replacement_rows = changes_df_filled[['internal_id', 'Q3_update'] + policy_columns].copy()
replacement_rows = replacement_rows.rename(columns={'Q3_update': 'response'})

# Merge in media_outlet from merged_df using internal_id
media_outlet_map = merged_df[['internal_id', 'media_outlet']].drop_duplicates()
replacement_rows = replacement_rows.merge(media_outlet_map, on='internal_id', how='left')

# Add a flag for annotator being absent since this is a post-hoc team update by KL and JOG
replacement_rows['annotator'] = None 

# Add any missing columns from merged_df
for col in merged_df.columns:
    if col not in replacement_rows.columns:
        replacement_rows[col] = None

# Reorder to match merged_df columns
replacement_rows = replacement_rows[merged_df.columns]

# Concatenate 
updated_df = pd.concat([filtered_merged_df, replacement_rows], ignore_index=True)
updated_df=updated_df.drop(columns= ['policy_y'])


Number of unique internal_ids to update: 16


In [113]:
merged_long = updated_df.melt(id_vars=['internal_id', 'media_outlet', 'response'],
                          value_vars=all_policies,
                          var_name='policy_category',
                          value_name='is_mentioned')

# Keep only policies that were mentioned
merged_long = merged_long[merged_long['is_mentioned'] == True]

In [114]:


merged_long['internal_id'] = merged_long['internal_id'].astype(int)

# Count unique internal_ids per (policy_category, media_outlet)
grouped = (
    merged_long.groupby(['policy_category', 'media_outlet'])['internal_id']
    .nunique()
    .reset_index()
)

# Pivot the table to have policies as rows, outlets as columns
policy_by_outlet = grouped.pivot_table(
    index='policy_category',
    columns='media_outlet',
    values='internal_id',
    fill_value=0
).astype(int)

policy_by_outlet.to_csv('output/policy_transcript_counts_by_outlet.csv', index=True)

In [115]:
naive_counts = pd.crosstab(index=merged_long['policy_category'],
                          columns=merged_long['response'])

In [116]:
naive_counts #non-normalized

response,Debate,Neutral,Opposes,Supports
policy_category,,,,
(De)regulation and laws,26,116,111,258
Emergency declaration,6,14,19,28
Emission reduction,49,203,195,622
Executive action,70,649,415,975
Green New Deal,24,90,247,197
Oil and gas industry,57,250,325,533
Other climate policy,31,122,59,215
Paris Agreement,38,341,178,598
Renewable energy,39,184,201,595


In [117]:
annotation_weighted_equally_counts = naive_counts/4

In [118]:
annotation_weighted_equally_counts

response,Debate,Neutral,Opposes,Supports
policy_category,,,,
(De)regulation and laws,6.50,29.00,27.75,64.50
Emergency declaration,1.50,3.50,4.75,7.00
Emission reduction,12.25,50.75,48.75,155.50
Executive action,17.50,162.25,103.75,243.75
Green New Deal,6.00,22.50,61.75,49.25
Oil and gas industry,14.25,62.50,81.25,133.25
Other climate policy,7.75,30.50,14.75,53.75
Paris Agreement,9.50,85.25,44.50,149.50
Renewable energy,9.75,46.00,50.25,148.75


In [119]:
# Reset index and rename the index column
annotation_weighted_equally_counts = annotation_weighted_equally_counts.reset_index().rename(columns={'response': 'policy_category'})

# Reorder columns
column_order = ['policy_category', 'Supports', 'Neutral', 'Debate', 'Opposes']
annotation_weighted_equally_counts_reordered = annotation_weighted_equally_counts[column_order]

# Save to CSV
annotation_weighted_equally_counts_reordered.to_csv('output/policy_valence_summary_annotation_weighing.csv', index=False)

## Set up code to weigh each transcript equally or each annotation equally

In [120]:
# Number of stance (Q3) annotators per transcript
q3_annotator_counts = Q3_df.groupby('internal_id')['internal_id'].count()

# Number of policy annotators per transcript
policy_annotator_counts = df_binary.groupby('internal_id')['internal_id'].count()

# Combine into a DataFrame to compute possible annotator pairings
annotator_combo_counts = pd.DataFrame({
    'q3_n': q3_annotator_counts,
    'policy_n': policy_annotator_counts
})
annotator_combo_counts['num_annotator_pairs'] = annotator_combo_counts['q3_n'] * annotator_combo_counts['policy_n']


In [121]:
# Remelt from merged_df, this time with a new name
melted_weighted = merged_df.melt(
    id_vars=['internal_id', 'media_outlet', 'response'],
    value_vars=all_policies,
    var_name='policy_category',
    value_name='is_mentioned'
)

# Keep only mentioned policies
melted_weighted = melted_weighted[melted_weighted['is_mentioned'] == True]

# Add number of annotator pairs per transcript
melted_weighted = melted_weighted.merge(
    annotator_combo_counts['num_annotator_pairs'],
    on='internal_id',
    how='left'
)

# Compute row weights
melted_weighted['weight'] = 1 / melted_weighted['num_annotator_pairs']


In [122]:

# Weighted sum grouped by policy and response
weighted_policy_response_table = (
    melted_weighted
    .groupby(['policy_category', 'response'])['weight']
    .sum()
    .unstack(fill_value=0)
)


In [123]:
weighted_policy_response_table

response,Debate,Neutral,Opposes,Supports
policy_category,,,,
(De)regulation and laws,8.50,34.00,30.00,68.00
Emergency declaration,1.50,4.00,5.50,7.00
Emission reduction,13.75,55.50,53.25,164.00
Executive action,20.00,179.75,111.75,258.00
Green New Deal,7.00,24.00,67.00,53.00
Oil and gas industry,15.75,73.75,88.75,141.75
Other climate policy,8.75,36.25,15.50,58.50
Paris Agreement,10.50,88.00,47.50,152.50
Renewable energy,11.25,55.50,54.75,156.50


In [124]:
# Reset index and rename the index column
weighted_policy_response_table = weighted_policy_response_table.reset_index().rename(columns={'response': 'policy_category'})

# Reorder columns
column_order = ['policy_category', 'Supports', 'Neutral', 'Debate', 'Opposes']
weighted_policy_response_table_reordered = weighted_policy_response_table[column_order]

# Save to CSV
weighted_policy_response_table_reordered.to_csv('output/policy_valence_summary_transcript_weighing.csv', index=False)
